In [1]:
def adataTostarmap(adata, cord, featureN, valuetoplot, result_path):
    if cord == 'umap':
        cord_xyz = pd.DataFrame(adata.obsm['X_umap'],columns=['x', 'y'])
        cord_xyz['z'] = 1
    if cord == 'pca':
        cord_xyz = pd.DataFrame(adata.obsm['X_pca'][:,0:3], columns = ['x', 'y', 'z'])
    else:
        cord_x = pd.DataFrame(adata.obsm['X_pca'][:,int(cord.split("_")[1][0])-1], columns = ['x'])
        cord_y = pd.DataFrame(adata.obsm['X_pca'][:,int(cord.split("_")[1][1])-1], columns = ['y'])
        cord_z = pd.DataFrame(adata.obsm['X_pca'][:,int(cord.split("_")[1][2])-1], columns = ['z'])
        cord_xyz = pd.concat([cord_x,cord_y], axis=1, join='inner')
        cord_xyz = pd.concat([cord_xyz,cord_z], axis=1, join='inner')
        
    fcolumns = []
    features = {}
    for i in range(len(featureN)):
        fcolumns.append('PC'+str(featureN[i]))
        features[i] = list(adata.obsm['X_pca'][:,featureN[i]])
    pcs = pd.DataFrame.from_dict(features)
    pcs.columns = fcolumns

    if valuetoplot in adata.obs.columns:
        label = pd.DataFrame(adata.obs[valuetoplot])
        label['cluster'] = label[valuetoplot]
        label = label.reset_index()
    else:
        label = pd.DataFrame(adata.raw.obs_vector(valuetoplot).round(),columns = ['cluster'])

    adataStarmap = pd.concat([cord_xyz,pcs], axis=1, join='inner')
    adataStarmap = pd.concat([adataStarmap,label['cluster']], axis=1, join='inner')
    adataStarmap.to_csv(result_path,index = None)

In [2]:
#Example 1:
#PC1,2,3 as 3D coordinates
#PC1,2,3,4,5,6,7,8,9,10,11,12 as features
#louvain clustering result as cluster labels
import scanpy as sc
import pandas as pd

adata = sc.read("~/scanpy_tutorial/write/pbmc3k.h5ad")
cord = 'pca_123'
featureN = [1,2,3,4,5,6,7,8,9,10,11,12]
valuetoplot = 'louvain'
result_path = '~/scanpy_tutorial/write/pbmc3k_louvain_starmap.csv'
adataTostarmap(adata, cord, featureN, valuetoplot, result_path)

In [3]:
#Example 1:
#PC2,3,5 as 3D coordinates
#PC3,5,6,7,15 as features
#Gene 'CPSF3L' expression level as cluster labels
import scanpy as sc
import pandas as pd

adata = sc.read("~/scanpy_tutorial/write/pbmc3k.h5ad")
cord = 'pca_235'
featureN = [3,5,6,7,15]
valuetoplot = 'CPSF3L'
result_path = '~/scanpy_tutorial/write/pbmc3k_CPSF3L_starmap.csv'
adataTostarmap(adata, cord, featureN, valuetoplot, result_path)